In [1]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [2]:
import torch
from torch.utils.data import DataLoader, TensorDataset

def make_dataloader(n):
    X = np.random.random((n, 2)) * 2 - 1
    Y = (X[:, 0] > X[:, 1] + 0.25).astype(int) + 1

    X = torch.tensor(X, dtype=torch.float)
    Y = torch.tensor(Y, dtype=torch.long)

    div1 = int(n*0.8)
    div2 = int(n*0.9)
    Xs = [X[:div1], X[div1:div2], X[div2:]]
    Ys = [Y[:div1], Y[div1:div2], Y[div2:]]

    dataset = TensorDataset(Xs[0], Ys[0])
    data_loader = DataLoader(dataset, batch_size=4)
    return data_loader

In [3]:
import torch.nn as nn
from metal.mmtl.task import Task

foo_input = nn.Linear(2, 10)
bar_input = foo_input #nn.Linear(100, 7)

foo_head = nn.Linear(10, 2)
bar_head = nn.Linear(10, 2)

foo_data = make_dataloader(5000)
bar_data = make_dataloader(2000)

foo = Task("foo_task", {"train": foo_data, "valid": foo_data}, foo_input, foo_head)
bar = Task("bar_task", {"train": bar_data, "valid": bar_data}, bar_input, bar_head)
# baz = Task("baz_task", "baz_head", [make_dataloader(100), None, None])
tasks = [foo, bar]

In [4]:
from metal.end_model import EndModel
from metal.mmtl.metal_model import MetalModel
from metal.mmtl.trainer import MultitaskTrainer

model = MetalModel(tasks, verbose=False)
trainer = MultitaskTrainer()
trainer.train_model(
    model, 
    tasks, 
    n_epochs=4, 
    lr=0.1, 
    progress_bar=True,
    log_every=1,
    score_every=2,
    checkpoint_best=True,
#     checkpoint_runway=2,
    checkpoint_metric="foo_task/valid/accuracy",
    checkpoint_metric_mode="max",
)

[1.0 epo]: TRAIN:[loss=0.101]



[2.0 epo]: TRAIN:[loss=0.055] VALID:[foo_task/accuracy=0.978, bar_task/accuracy=0.955]
Saving model at iteration 2.0 with best (max) score 0.978



[3.0 epo]: TRAIN:[loss=0.094]

Restoring best model from iteration 2.0 with score 0.978
Finished Training
{'bar_task/valid/accuracy': 0.955, 'foo_task/valid/accuracy': 0.9775}


In [7]:
for batch in foo.data_loaders["train"]:
    X, Y = batch
    print(model(X, ['foo_task']))
    print(model.calculate_loss(X, Y, ['foo_task']))    
    print(model.calculate_output(X, ['foo_task']))    
    break

{'foo_task': tensor([[ 38.4949, -34.9645],
        [-34.9939,  31.0529],
        [ -0.9784,   0.6648],
        [-35.1590,  31.1351]], grad_fn=<AddmmBackward>)}
{'foo_task': tensor(0.0442, grad_fn=<NllLossBackward>)}
{'foo_task': tensor([[1.0000e+00, 1.2502e-32],
        [2.0712e-29, 1.0000e+00],
        [1.6202e-01, 8.3798e-01],
        [1.6175e-29, 1.0000e+00]])}


In [ ]:
next(model.parameters()).device